In [1]:
from run_re import InputExample, convert_single_example,model_fn_builder, file_based_input_fn_builder
import modeling
import tokenization
import tensorflow as tf
import os

/opt/conda/envs/biobert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/biobert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/biobert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/biobert/lib/python3.6/site-packages/tensorflow/python/framework

# Read example and convert to features

In [2]:
guid = 0
text_a = "These results suggest that the C1772T polymorphism in @GENE$ is not involved in progression or metastasis of @DISEASE$"
label = 1

In [3]:
vocab_file="/home/jupyter/BioBERT/biobert_base/vocab.txt"

In [4]:
tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=False)

In [5]:
example = InputExample(guid=guid, text_a=text_a,label=label)

In [6]:
feature = convert_single_example(ex_index=0,example=example, 
                                 label_list=[0,1], max_seq_length=128, tokenizer=tokenizer)

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] These results suggest that the C ##17 ##7 ##2 ##T p ##oly ##mor ##phism in @ GE ##NE $ is not involved in progression or meta ##sta ##sis of @ D ##IS ##EA ##SE $ [SEP]
INFO:tensorflow:input_ids: 101 1636 2686 5996 1115 1103 140 16770 1559 1477 1942 185 23415 26271 19649 1107 137 25075 22680 109 1110 1136 2017 1107 16147 1137 27154 8419 4863 1104 137 141 6258 12420 12649 109 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [7]:
feature.label_id

1

# Load trained model

In [9]:
bert_config = modeling.BertConfig.from_json_file("/home/jupyter/BioBERT/biobert_base/bert_config.json")

In [26]:
run_config = tf.contrib.tpu.RunConfig(
      cluster=None,
      model_dir='re_outputs_1',
      save_checkpoints_steps=1000)

In [12]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=2,
      init_checkpoint='re_outputs_2/model.ckpt-449',
      learning_rate=5e-5,
      num_train_steps=0,
      num_warmup_steps=0,
      use_tpu=False,
      use_one_hot_embeddings=False)

In [27]:
estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=False,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=1,
      eval_batch_size=1,
      predict_batch_size=1)

INFO:tensorflow:Using config: {'_model_dir': 're_outputs_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe9cdb6e4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_

# Make prediction from file

In [28]:
predict_file = os.path.join('re_outputs_1', "predict.tf_record")

In [29]:
predict_input_fn = file_based_input_fn_builder(
        input_file=predict_file,
        seq_length=128,
        is_training=False,
        drop_remainder=False)

In [30]:
result = estimator.predict(input_fn=predict_input_fn)

In [31]:
list(result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (28996, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *

[{'probabilities': array([0.02633333, 0.9736667 ], dtype=float32)},
 {'probabilities': array([0.30977336, 0.6902266 ], dtype=float32)},
 {'probabilities': array([0.9779052 , 0.02209483], dtype=float32)},
 {'probabilities': array([0.2272355, 0.7727645], dtype=float32)},
 {'probabilities': array([0.04970019, 0.9502998 ], dtype=float32)},
 {'probabilities': array([0.01793885, 0.98206115], dtype=float32)},
 {'probabilities': array([0.04228313, 0.95771694], dtype=float32)},
 {'probabilities': array([0.09953282, 0.90046716], dtype=float32)},
 {'probabilities': array([0.05416417, 0.94583577], dtype=float32)},
 {'probabilities': array([0.03082338, 0.9691766 ], dtype=float32)},
 {'probabilities': array([0.05237907, 0.947621  ], dtype=float32)},
 {'probabilities': array([0.06347434, 0.93652564], dtype=float32)},
 {'probabilities': array([0.03322292, 0.9667771 ], dtype=float32)},
 {'probabilities': array([0.03306265, 0.96693736], dtype=float32)},
 {'probabilities': array([0.03671705, 0.96328294],